In [14]:
import os

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from peft import PeftModel, PeftConfig
import torch

In [ ]:
base_model = "mistralai/Mistral-Nemo-Instruct-2407"
adapter_model = "shikhac30/mistral_nemo_12b_medical_ft_v1"
hf_token = "replace_hf_token"

In [18]:
final_model = "mistral_nemo_12b_medical_model"  


In [9]:


# 1. Load tokenizer and config from base model
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
config = AutoConfig.from_pretrained(base_model)

# 2. Load base model in 16-bit
base = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# 3. Load adapter and apply
model = PeftModel.from_pretrained(base, adapter_model)
model = model.merge_and_unload()  # merge LoRA into base weights



Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/514M [00:00<?, ?B/s]

In [17]:

local_path = "./merged_model"
os.makedirs(local_path, exist_ok=True)

model.save_pretrained(local_path, safe_serialization=True)
tokenizer.save_pretrained(local_path)



In [20]:
%time
# Push to HF
model.push_to_hub(final_model, token=hf_token)
tokenizer.push_to_hub(final_model, token=hf_token)

CPU times: user 25 μs, sys: 11 μs, total: 36 μs
Wall time: 69.9 μs


model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shikhac30/mistral_nemo_12b_medical_model/commit/792d6264f158e18df1140ff439750d71b0baaa4b', commit_message='Upload tokenizer', commit_description='', oid='792d6264f158e18df1140ff439750d71b0baaa4b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shikhac30/mistral_nemo_12b_medical_model', endpoint='https://huggingface.co', repo_type='model', repo_id='shikhac30/mistral_nemo_12b_medical_model'), pr_revision=None, pr_num=None)